In [ ]:
!pip install emoji

In [ ]:
!pip install keras

In [ ]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
data = pd.read_csv("emoji_data.csv", header = None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [ ]:
emoji_dict = {
    0 : ":red_heart:",
    1 : ":baseball:",
    2 : ":grinning_face_with_big_eyes:",
    3 : ":disappointed_face:",
    4 : ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):

  return emoji.emojize(emoji_dict[label])

In [ ]:
X = data[0].values
Y = data[1].values

In [ ]:
X

array(['French macaroon is so tasty', 'work is horrible', 'I am upset',
       'throw the ball', 'Good joke',
       'what is your favorite baseball game', 'I cooked meat',
       'stop messing around', 'I want chinese food',
       'Let us go play baseball', 'you are failing this exercise',
       'yesterday we lost again', 'Good job', 'ha ha ha it was so funny',
       'I will have a cheese cake', 'Why are you feeling bad',
       'I want to joke', 'I never said yes for this',
       'the party is cancelled', 'where is the ball', 'I am frustrated',
       'ha ha ha lol', 'she said yes', 'he got a raise',
       'family is all I have', 'he can pitch really well',
       'I love to the stars and back', 'do you like pizza ',
       'You totally deserve this prize', 'I miss you so much',
       'I like your jacket ', 'she got me a present',
       'will you be my valentine', 'you failed the midterm',
       'Who is down for a restaurant', 'valentine day is near',
       'Great so awesome

In [ ]:
Y

array([4, 3, 3, 1, 2, 1, 4, 3, 4, 1, 3, 3, 2, 2, 4, 3, 2, 3, 3, 1, 3, 2,
       2, 2, 0, 1, 0, 4, 2, 3, 2, 0, 0, 3, 4, 0, 2, 1, 3, 1, 0, 4, 0, 3,
       0, 4, 2, 3, 4, 2, 2, 3, 0, 2, 2, 3, 2, 3, 2, 2, 3, 3, 0, 2, 3, 0,
       2, 0, 0, 2, 3, 2, 4, 1, 3, 3, 0, 0, 3, 2, 0, 3, 0, 2, 2, 4, 2, 2,
       0, 0, 2, 3, 0, 4, 2, 1, 2, 3, 3, 2, 3, 0, 3, 0, 2, 0, 2, 3, 4, 3,
       1, 3, 4, 3, 2, 3, 3, 3, 1, 4, 4, 2, 2, 1, 1, 2, 3, 2, 3, 4, 2, 3,
       0, 2, 0, 0, 4, 3, 4, 2, 3, 2, 3, 4, 2, 1, 2, 4, 3, 1, 3, 2, 3, 2,
       2, 3, 3, 2, 4, 0, 0, 0, 3, 0, 0, 1, 1, 2, 2, 2, 0, 3, 2, 3, 3, 1,
       2, 2, 4, 2, 3, 1, 2])

# **Embeddings**

In [ ]:
file = open("glove.6B.100d.txt", "r", encoding = "utf8")

content = file.readlines()
file.close()

In [ ]:
embeddings = {}

for line in content[:2]:
  line = line.split()
  print(line)

['the', '-0.038194', '-0.24487', '0.72812', '-0.39961', '0.083172', '0.043953', '-0.39141', '0.3344', '-0.57545', '0.087459', '0.28787', '-0.06731', '0.30906', '-0.26384', '-0.13231', '-0.20757', '0.33395', '-0.33848', '-0.31743', '-0.48336', '0.1464', '-0.37304', '0.34577', '0.052041', '0.44946', '-0.46971', '0.02628', '-0.54155', '-0.15518', '-0.14107', '-0.039722', '0.28277', '0.14393', '0.23464', '-0.31021', '0.086173', '0.20397', '0.52624', '0.17164', '-0.082378', '-0.71787', '-0.41531', '0.20335', '-0.12763', '0.41367', '0.55187', '0.57908', '-0.33477', '-0.36559', '-0.54857', '-0.062892', '0.26584', '0.30205', '0.99775', '-0.80481', '-3.0243', '0.01254', '-0.36942', '2.2167', '0.72201', '-0.24978', '0.92136', '0.034514', '0.46745', '1.1079', '-0.19358', '-0.074575', '0.23353', '-0.052062', '-0.22044', '0.057162', '-0.15806', '-0.30798', '-0.41625', '0.37972', '0.15006', '-0.53212', '-0.2055', '-1.2526', '0.071624', '0.70565', '0.49744', '-0.42063', '0.26148', '-1.538', '-0.30223

In [ ]:
for line in content:
  line = line.split()
  embeddings[line[0]] = np.array(line[1:], dtype = float)

In [ ]:
#embeddings

In [ ]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

In [ ]:
Xtokens = tokenizer.texts_to_sequences(X)

In [ ]:
def get_maxlen(data):
  maxlen = 0

  for sentence in data:
    maxlen = max(maxlen, len(sentence))

  return maxlen

maxlen = get_maxlen(Xtokens)
print(maxlen)

10


In [ ]:
xtrain = pad_sequences(Xtokens,
                       maxlen = maxlen,
                       padding = "post",
                       truncating = "post")

In [ ]:
xtrain[:2]

array([[103, 104,   3,   6, 105,   0,   0,   0,   0,   0],
       [106,   3, 107,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [ ]:
ytrain = to_categorical(Y)

# **Model**

In [ ]:
embd_size = 100

embedding_matrix = np.zeros((len(word2index) + 1, embd_size))

for word, i in word2index.items():
    if word in embeddings:
        embd_vector = embeddings[word]
        embedding_matrix[i] = embd_vector
    else:
        # Handle the case when the word is not found in embeddings
        # For example, you might assign a random vector or use a special token.
        pass

In [ ]:
embedding_matrix.shape

(313, 100)

In [ ]:
model = Sequential([
    Embedding(input_dim = len(word2index)+1,
              output_dim = embd_size,
              input_length = maxlen,
              weights = [embedding_matrix],
              trainable = False),

    LSTM(units = 16, return_sequences = True),
    LSTM(units = 4),
    Dense(units = 5, activation = "softmax")
])

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 100)           31300     
                                                                 
 lstm_2 (LSTM)               (None, 10, 16)            7488      
                                                                 
 lstm_3 (LSTM)               (None, 4)                 336       
                                                                 
 dense_1 (Dense)             (None, 5)                 25        
                                                                 
Total params: 39,149
Trainable params: 7,849
Non-trainable params: 31,300
_________________________________________________________________


In [ ]:
model.fit(xtrain, ytrain, epochs = 100)

Epoch 1/100
6/6 [==============================] - 6s 20ms/step - loss: 1.6024 - accuracy: 0.2787
Epoch 2/100
6/6 [==============================] - 0s 19ms/step - loss: 1.5814 - accuracy: 0.2732
Epoch 3/100
6/6 [==============================] - 0s 19ms/step - loss: 1.5653 - accuracy: 0.2951
Epoch 4/100
6/6 [==============================] - 0s 21ms/step - loss: 1.5515 - accuracy: 0.3005
Epoch 5/100
6/6 [==============================] - 0s 20ms/step - loss: 1.5394 - accuracy: 0.3115
Epoch 6/100
6/6 [==============================] - 0s 18ms/step - loss: 1.5266 - accuracy: 0.3060
Epoch 7/100
6/6 [==============================] - 0s 18ms/step - loss: 1.5116 - accuracy: 0.3115
Epoch 8/100
6/6 [==============================] - 0s 20ms/step - loss: 1.4926 - accuracy: 0.3333
Epoch 9/100
6/6 [==============================] - 0s 18ms/step - loss: 1.4681 - accuracy: 0.3716
Epoch 10/100
6/6 [==============================] - 0s 17ms/step - loss: 1.4357 - accuracy: 0.4699
Epoch 11/100
6/6 [=

In [ ]:
test = ["I feel good", "I feel very bad", "Lets eat dinner", "Hey", "Thank you", "Dinner Time"]

test_seq = tokenizer.texts_to_sequences(test)
xtest = pad_sequences(test_seq, maxlen = maxlen, padding = "post", truncating = "post")

y_pred = model.predict(xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(xtest)):
  print(test[i], label_to_emoji(y_pred[i]))

1/1 [==============================] - 0s 74ms/step
I feel good 😃
I feel very bad 😞
Lets eat dinner 🍽️
Hey 😃
Thank you 😃
Dinner Time 🍽️
